In [1]:
import pandas as pd
import sys
sys.path.append('../')
from datasets.crsp_data import CRSP
from datasets.compustat import COMPUSTAT as CSTAT

### Use synthetic control and diff in diff and interrupted time series
For synthetic control:
- Create a counterfactual company based on observables

For diff in diff:
- Identify a single pair company

For interrupted TS:
- Compute change in excess return

Question: Excess returns or change in absolute returns?

In [24]:
pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-27/departures.csv')

,dismissal_dataset_id,coname,gvkey,fyear,co_per_rol,exec_fullname,departure_code,ceo_dismissal,interim_coceo,tenure_no_ceodb,max_tenure_ceodb,fyear_gone,leftofc,still_there,notes,sources,eight_ks,cik,_merge
0,559043,SONICBLUE INC,27903,2002,-1,L. Gregory Ballard,7.0,0.0,NaN,1,1,2003.0,2003-03-21T00:00:00Z,NaN,Ballard took over when the outgoing CEO said t...,https://www.wsj.com/articles/SB102885769219093...,https://www.sec.gov/Archives/edgar/data/850519...,850519.0,matched (3)
1,12,AMERICAN AIRLINES GROUP INC,1045,1997,1,Robert L. Crandall,5.0,0.0,NaN,1,1,1998.0,1998-05-20T00:00:00Z,NaN,"Mr. Crandall, the chairman and chief executive...",https://www.nytimes.com/1998/04/16/business/co...,https://www.sec.gov/Archives/edgar/data/6201/0...,6201.0,matched (3)
2,13,AMERICAN AIRLINES GROUP INC,1045,2002,3,Donald J. Carty,3.0,1.0,NaN,1,1,2003.0,2003-04-24T00:00:00Z,NaN,Donald Carty resigned as the head of American ...,https://www.chron.com/business/article/America...,https://www.sec.gov/Archives/edgar/data/6201/0...,6201.0,matched (3)
3,31,ABBOTT LABORATORIES,1078,1998,6,Duane L. Burnham,5.0,0.0,NaN,1,1,1998.0,1998-12-31T00:00:00Z,NaN,The top two executives of Abbott Laboratories ...,https://www.wsj.com/articles/SB884916766271843500,NaN,1800.0,matched (3)
4,43,ADVANCED MICRO DEVICES,1161,2001,11,Walter Jeremiah Sanders III,5.0,0.0,NaN,1,1,2002.0,2002-04-25T00:00:00Z,NaN,Advanced Micro Devices founder and chairman Wa...,https://www.computerweekly.com/news/2240054556...,https://www.sec.gov/Archives/edgar/data/2488/0...,2488.0,matched (3)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9418,7998,ZYNEX INC,129441,2019,64584,Thomas Sandgaard,NaN,NaN,NaN,1,1,NaN,NaN,probably,NaN,NaN,NaN,846475.0,matched (3)
9419,8906,HUDSON PACIFIC PPTYS INC,185018,2019,64594,Victor J. Coleman,NaN,NaN,NaN,1,1,NaN,NaN,probably,NaN,NaN,NaN,1482512.0,matched (3)
9420,8935,FRANCESCAS HOLDINGS CORP,187041,2019,64596,Michael D. Prendergast,NaN,NaN,NaN,1,1,2020.0,2020-03-09T00:00:00Z,NaN,NaN,NaN,https://www.sec.gov/Archives/edgar/data/139993...,1399935.0,matched (3)
9421,8968,GROCERY OUTLET HLDNG CORP,189579,2018,64601,"S. MacGregor Read, Jr.",5.0,0.0,CO-CEO,1,1,2018.0,2018-12-01T00:00:00Z,NaN,"Read, the grandson of Grocery Outlet founder J...",https://www.globenewswire.com/news-release/202...,NaN,1771515.0,matched (3)


In [26]:
ceo_deaths = pd.read_csv('https://raw.githubusercontent.com/rfordatascience/tidytuesday/master/data/2021/2021-04-27/departures.csv')
ceo_deaths = ceo_deaths[ceo_deaths['departure_code'] == 1]
ceo_deaths.dropna(how='any', inplace=True, axis=1)
ceo_deaths = ceo_deaths[['gvkey', 'fyear', 'exec_fullname', 'sources']]
ceo_deaths.head()

,gvkey,fyear,exec_fullname,sources
169,3144,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...
218,3734,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...
316,5151,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...
440,7154,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...
525,8151,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...


In [27]:
crsp = CRSP()
crsp.df.head()

LOADING CLEAN FILE


,permno,date,cusip,shrcd,exchcd,ticker,shrout,vol,prc,ret
0,10000,1986-02-28,68391610,10.0,3.0,OMFGA,3680.0,828.0,3.25000,-0.257143
1,10000,1986-03-31,68391610,10.0,3.0,OMFGA,3680.0,1078.0,4.43750,0.365385
2,10000,1986-04-30,68391610,10.0,3.0,OMFGA,3793.0,957.0,4.00000,-0.098592
3,10000,1986-05-30,68391610,10.0,3.0,OMFGA,3793.0,1074.0,3.10938,-0.222656
4,10000,1986-06-30,68391610,10.0,3.0,OMFGA,3793.0,1069.0,3.09375,-0.005025


In [28]:
cstat = CSTAT()
cstat.df.head()

LOADING CLEAN FILE


,gvkey,datadate,fyearq,fqtr,fyr,indfmt,consol,popsrc,datafmt,tic,...,priusa,sic,spcindcd,spcseccd,spcsrc,state,stko,weburl,dldte,ipodate
0,1000,1966-03-31,1966,1.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
1,1000,1966-06-30,1966,2.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
2,1000,1966-09-30,1966,3.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
3,1000,1966-12-31,1966,4.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None
4,1000,1967-03-31,1967,1.0,12,INDL,C,D,STD,AE.2,...,1.0,3089,325.0,978.0,None,None,0.0,None,1978-06-30,None


In [29]:
linking_df = cstat.df[['gvkey', 'cusip', 'fyearq']].drop_duplicates()
ceo_deaths = pd.merge(ceo_deaths, linking_df, left_on=['gvkey', 'fyear'], right_on=['gvkey', 'fyearq'], how='left')
ceo_deaths.head()

,gvkey,fyear,exec_fullname,sources,cusip,fyearq
0,3144,1997,Roberto C. Goizueta,https://www.washingtonpost.com/archive/local/1...,191216100,1997
1,3734,2003,Joseph M. Magliochetti,https://www.toledoblade.com/frontpage/2003/09/...,235825205,2003
2,5151,1995,Israel Cohen,https://www.baltimoresun.com/news/bs-xpm-1995-...,374478105,1995
3,7154,2003,James R. Cantalupo,https://money.cnn.com/2004/04/19/news/fortune5...,580135101,2003
4,8151,2000,Larry W. Brummett,https://newsok.com/article/2709352/chief-execu...,682680103,2000


In [30]:
for source in ceo_deaths['sources'].unique():
    print(source)
    

https://www.washingtonpost.com/archive/local/1997/10/19/coca-cola-chief-roberto-goizueta-dies/9bb4d612-91d1-44df-95ca-5ddeaf95d345/
https://www.toledoblade.com/frontpage/2003/09/24/Dana-Corp-chairman-61-dies-of-pancreas-ailment.html
 https://www.chicagotribune.com/news/ct-xpm-2003-09-26-0309260231-story.html
 https://www.toledoblade.com/frontpage/2003/09/24/Dana-Corp-chairman-61-dies-of-pancreas-ailment.html
 http://www.landlinemag.com/story.aspx?storyid=43889
 https://www.google.com/search?safe=active&rlz=1C1CHBF_enUS819US819&tbs=cdr:1,cd_min:05/24/2003,cd_max:01/21/2004&tbm=nws&q=Joseph+Magliochetti+DANA&source=lnms&tbm=nws&sa=X&ved=0ahUKEwj-o8-h0__hAhUJP6wKHX8eCbEQ_AUIDigB&biw=1920&bih=969
 https://www.google.com/search?safe=active&rlz=1C1CHBF_enUS819US819&tbs=cdr:1,cd_min:03/26/2003,cd_max:12/21/2003&tbm=nws&q=Joseph+Magliochetti+DANA
 https://www.google.com/search?q=Joseph+Magliochetti+DANA
https://www.baltimoresun.com/news/bs-xpm-1995-11-24-1995328013-story.html
https://money.cnn